## BERT

In [18]:
## for data
import json
import pandas as pd
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, SpatialDropout1D
from keras.layers.recurrent import LSTM
from keras.layers import Activation, Dense
from keras.callbacks import ModelCheckpoint, EarlyStopping
## for plotting
import matplotlib.pyplot as plt
import seaborn as sns

## for processing
import re
import nltk
from nltk.tokenize import word_tokenize
## for bag-of-words
from sklearn import feature_extraction, model_selection, naive_bayes, pipeline, manifold, preprocessing, feature_selection, metrics

## for explainer
from lime import lime_text

## for word embedding
import gensim
import gensim.downloader as gensim_api

## for deep learning
from tensorflow.keras import models, layers, preprocessing as kprocessing
from tensorflow.keras import backend as K

## for bert language model
import transformers

In [19]:
#!pip install torch

In [20]:
import pandas as pd
dtf = pd.read_csv("data/data_cleaned.csv")

In [21]:
dtf_new = dtf.rename(columns={'category': 'labels', 'text_clean': 'text'})

In [22]:
dtf_new

,Unnamed: 0,headline,labels,text
0,1,Will Smith Joins Diplo And Nicky Jam For The 2...,ENTERTAINMENT,smith join diplo nicky jam world cup official ...
1,2,Hugh Grant Marries For The First Time At Age 57,ENTERTAINMENT,hugh grant marries first time age
2,3,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,ENTERTAINMENT,jim carrey blast castrato adam schiff democrat...
3,4,Julianna Margulies Uses Donald Trump Poop Bags...,ENTERTAINMENT,julianna margulies us donald trump poop bag pi...
4,5,Morgan Freeman 'Devastated' That Sexual Harass...,ENTERTAINMENT,morgan freeman devastated sexual harassment cl...
...,...,...,...,...
58989,200844,Google+ Now Open for Teens With Some Safeguards,SCIENCE&TECH,google open teen safeguard
58990,200845,Web Wars,SCIENCE&TECH,web war
58991,200846,"First White House Chief Technology Officer, An...",SCIENCE&TECH,first white house chief technology officer ane...
58992,200847,Watch The Top 9 YouTube Videos Of The Week,SCIENCE&TECH,watch top youtube video week


In [23]:
from sklearn.preprocessing import LabelEncoder

lb = LabelEncoder()
dtf_new["labels"] = lb.fit_transform(dtf_new["labels"])

In [24]:
data = dtf_new.drop("headline", axis=1)

In [25]:
data

,Unnamed: 0,labels,text
0,1,1,smith join diplo nicky jam world cup official ...
1,2,1,hugh grant marries first time age
2,3,1,jim carrey blast castrato adam schiff democrat...
3,4,1,julianna margulies us donald trump poop bag pi...
4,5,1,morgan freeman devastated sexual harassment cl...
...,...,...,...
58989,200844,3,google open teen safeguard
58990,200845,3,web war
58991,200846,3,first white house chief technology officer ane...
58992,200847,3,watch top youtube video week


In [26]:
lb.classes_

array(['BUSINESS', 'ENTERTAINMENT', 'POLITICS', 'SCIENCE&TECH'],
      dtype=object)

In [27]:
data['labels']

0        1
1        1
2        1
3        1
4        1
        ..
58989    3
58990    3
58991    3
58992    3
58993    3
Name: labels, Length: 58994, dtype: int32

In [28]:
X = data

In [29]:
X_train, X_test = model_selection.train_test_split(X, test_size = 0.40, random_state = 42)
print(X_train.shape)
print(X_test.shape)

(35396, 3)
(23598, 3)


In [38]:
import pandas as pd

from simpletransformers.classification import ClassificationModel

# Train and Evaluation data needs to be in a Pandas Dataframe containing at least two columns. 
#If the Dataframe has a header, it should contain a 'text' and a 'labels' column. 
#If no header is present, the Dataframe should contain at least two columns, 
#with the first column is the text with type str, and the second column in the label with type int.
train_data = [
    ["Example sentence belonging to class 1", 1],
    ["Example sentence belonging to class 0", 0],
    ["Example eval senntence belonging to class 2", 2],
]
train_df = pd.DataFrame(train_data)

eval_data = [
    ["Example eval sentence belonging to class 1", 1],
    ["Example eval sentence belonging to class 0", 0],
    ["Example eval senntence belonging to class 2", 2],
]
eval_df = pd.DataFrame(eval_data)

# Create a ClassificationModel
model = ClassificationModel(
    "bert",
    "bert-base-cased",
    num_labels=4,
    args={"reprocess_input_data": True, "overwrite_output_dir": True},
    use_cuda=False
    
)

# Train the model
#model.train_model(X_train)

# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(X_test)

predictions, raw_outputs = model.predict(["Some arbitary sentence"])

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

  0%|          | 0/35396 [00:00<?, ?it/s]

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/4425 [00:00<?, ?it/s]

/home/dee/TEXT CLASSIFICATION PROJECT/simpletransformers/classification/classification_model.py:1401: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/3 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [13]:
#import torch
#torch.save(model, "model/model")

In [37]:
import torch
model = torch.load('model/model')

In [51]:
a = list(X_test["text"].iloc[0:5000])

In [52]:
predictions, raw_outputs = model.predict(a)

In [53]:
from sklearn.metrics import classification_report

print(classification_report(np.array(X_test["labels"].iloc[0:5000]), predictions))

              precision    recall  f1-score   support

           0       0.74      0.68      0.71       534
           1       0.90      0.90      0.90      1291
           2       0.92      0.94      0.93      2800
           3       0.77      0.72      0.74       375

    accuracy                           0.89      5000
   macro avg       0.83      0.81      0.82      5000
weighted avg       0.88      0.89      0.88      5000

